# MILESTONE 2

In [86]:
import numpy as np
import pandas as pd

import findspark
findspark.init()

from pyspark.sql import *

spark = SparkSession.builder.getOrCreate()

In [88]:
df_spark = spark.read.json("data samples/2007_12.json")
df_pandas = pd.read_json("data samples/2007_12.json", lines=True)

In [89]:
df_spark.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [90]:
df_pandas.head()

,archived,author,body,controversiality,created_utc,downs,edited,gilded,id,link_id,name,parent_id,retrieved_on,score,score_hidden,subreddit,subreddit_id,ups
0,True,[deleted],The fact remains that Chavez does not understa...,0,1196467200,0,false,0,c02khd5,t3_61tiq,t1_c02khd5,t1_c02kdcg,1427421031,1,False,politics,t5_2cneq,1
1,True,[deleted],[deleted],0,1196467207,0,false,0,c02khd6,t3_61u0v,t1_c02khd6,t3_61u0v,1427421031,0,False,politics,t5_2cneq,0
2,True,[deleted],"if reddit or anybody in the conde nast family,...",0,1196467235,0,false,0,c02khdb,t3_61twk,t1_c02khdb,t1_c02kh9l,1427421031,2,False,politics,t5_2cneq,2
3,True,[deleted],"Hillary, like most other candidates, has an or...",0,1196467251,0,false,0,c02khdj,t3_61u2f,t1_c02khdj,t1_c02kesz,1427421031,1,False,politics,t5_2cneq,1
4,True,[deleted],"""If Lenin walked around the offices of a compa...",0,1196467272,0,false,0,c02khdl,t3_61u0v,t1_c02khdl,t1_c02kfhk,1427421031,4,False,politics,t5_2cneq,4
